# Granularity in mixed-integer nonlinear optimization

## Summary of the document

This document is complementary to the paper "Granularity in mixed-integer nonlinear optimization". The intention is to make the numerical results transparent. The generated results for the IPCP, the feasibility pump as well as for B-Hyb are stored as pandas data-frames and are concatenated herein. 

The latex-tables of the article are produced by using the command print(dataframe.to_latex(float_format = '%.2f'))

In [39]:
import pandas as pd
pd.options.display.float_format = '${:,.2f}'.format
import numpy as np
import pickle
from ast import literal_eval

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

def store_FRA_results_for_Bonmin(pseudo_granular_results):
    np.save('vals_SOR',np.array(pseudo_granular_results['obj']))
    testbed = list(pseudo_granular_results.index)
    with open(r'../testbed/SOR_succ.txt', 'w') as f:
        for item in testbed:
            f.write("%s\n" % item)
def load_successful_results(name):
    results = load_obj(name)
    succ_results = results[results['obj'] != float('inf')]
    print(len(succ_results)," Problems are pseudo granular")
    return succ_results

def load_problems_with_computable_Lipschitz_constant(name):
    results = load_obj(name)
    L_results = results.loc[results['time L']<=1800]
    print("We can compute Lipschitz constants for", len(L_results)," Problems")
    return L_results

def load_problems_with_time_limit_Lipschitz(name):
    results = load_obj(name)
    L_results = results.loc[results['time L']>1800]
    print("We can't compute Lipschitz constants for", len(L_results)," Problems")
    return L_results
    
def find_simple_problems(result_frame):
    constr_quadrouple = result_frame['constrs'].to_list()
    simple_problems = []
    for idx, quadrouple in enumerate(constr_quadrouple):
        if quadrouple[-6:-1] == ' 0, 0':
            simple_problems.append(result_frame.index[idx])
    return simple_problems

def filter_instances_from_A_not_contained_in_B(resA,resB):
    res_set = set(resA.index)
    res_set.difference_update(list(resB.index))
    res_list = list(res_set)
    res_list.sort()
    return res_list

def filter_instances_nonlinear_in_y(result):
    result_nonlinear = result[result['constrs'].apply(lambda x: int(x[-2]))>0]
    return result_nonlinear

def filter_out_only_epi_problems(result):
    result_nonepi = result[result['constrs']!='(1, 1, 1, 1)']
    return result_nonepi

def update_objective_value_epi_problems(result):
    epi_subset = result['constr_value'] != float('-inf') 
    result.loc[epi_subset,'obj'] = np.array(result[epi_subset]['obj']+result[epi_subset]['constr_value'])
    result = result.drop('constr_value',axis=1)
    return result

def print_numbers_of_different_problemclasses(results):
    class0 = 0
    class1 = 0
    class2 = 0
    for name in results_overall.index.to_list():
        constr_tuple = literal_eval(results_overall.loc[name].constrs)
        if constr_tuple[2] == 0:
            class0 +=1
        elif constr_tuple[3] == 0:
            class1 +=1
        else:
            class2 += 1
    print(class0,class1,class2)
    
def print_numbers_of_purely_epi_problems(results):
    epi_problems = 0
    for name in results_overall.index.to_list():
        constr_array = np.array(literal_eval(results_overall.loc[name].constrs))
        if all(constr_array == 1):
            epi_problems +=1
    print(epi_problems)

## Analysis of results
We tested FRA-SOR for $\delta \in \{0.5,0.75,0.9999\}$, with $\delta$ being the enlargement parameter for the box constraints. We found that most problems were granular for $\delta = 0.9999$ (even though Lipschitz constants were bigger). However, interestingly, we found that the larger the value of $\delta$, the more difficult the computation of Lipschitz constants (i.e. the longer the solver needed to solve the auxiliary problem).

Next, we report our main results with $\delta = 0.9999$.

In [3]:
filename = 'SOR_all_instances_0.9999'
results_overall = load_obj(filename)
results_L_computable = load_problems_with_computable_Lipschitz_constant(filename)
results_pseudo_granular = load_successful_results(filename)
results_L_not_computable = load_problems_with_time_limit_Lipschitz(filename)

We can compute Lipschitz constants for 183  Problems
52  Problems are pseudo granular
We can't compute Lipschitz constants for 27  Problems


So we obtain feasible points for 52 optimization problems with $\delta = 0.9999$. Let us initially analyze the difficulty of the auxiliary problem.

In [40]:
print_numbers_of_different_problemclasses(results_overall)
print_numbers_of_purely_epi_problems(results_overall)

68 10 132
24


### Lipschitz computation
Here, we analyse how much time the computation of Lipschitz constants take dependently on the problem-size. Let us start with problems where the Lipschitz constant is not computable

In [4]:
results_L_not_computable[['vars','constrs']]

,vars,constrs
edgecross14-058,"(183, 182, 182)","(1457, 1, 1, 1)"
edgecross14-098,"(183, 182, 182)","(1457, 1, 1, 1)"
edgecross14-117,"(183, 182, 182)","(1457, 1, 1, 1)"
edgecross14-137,"(183, 182, 182)","(1457, 1, 1, 1)"
edgecross20-040,"(381, 380, 380)","(4561, 1, 1, 1)"
edgecross20-080,"(381, 380, 380)","(4561, 1, 1, 1)"
edgecross22-048,"(463, 462, 462)","(6161, 1, 1, 1)"
edgecross22-096,"(463, 462, 462)","(6161, 1, 1, 1)"
edgecross24-057,"(553, 552, 552)","(8097, 1, 1, 1)"
edgecross24-115,"(553, 552, 552)","(8097, 1, 1, 1)"


Below, we list the 20 interesting problems where the computation of all Lipschitz constants is possible and takes at least 2 seconds time

In [6]:
results_L_computable[results_L_computable['time L']>2][['vars','constrs','time L']]

,vars,constrs,time L
cvxnonsep_normcon40r,"(80, 20, 0)","(41, 40, 20, 20)",$2.34
edgecross10-040,"(91, 90, 90)","(481, 1, 1, 1)",$26.07
edgecross10-050,"(91, 90, 90)","(481, 1, 1, 1)",$51.41
edgecross10-070,"(91, 90, 90)","(481, 1, 1, 1)","$1,095.80"
edgecross14-176,"(183, 182, 182)","(1457, 1, 1, 1)",$240.85
sonet17v4,"(136, 136, 136)","(2057, 17, 17, 17)",$2.34
sonet18v6,"(153, 153, 153)","(2466, 18, 18, 18)",$2.55
sonet19v5,"(171, 171, 171)","(2926, 19, 19, 19)",$2.81
sonet20v6,"(190, 190, 190)","(3440, 20, 20, 20)",$3.06
sonet21v6,"(210, 210, 210)","(4011, 21, 21, 21)",$3.30


Interestingly, as mentioned above, for enlargement parameter $\delta = 0.5$, we can compute Lipschitz constants even for larger problems. Below, we list all 14 problems where we get a Lipschitz constant for $\delta = 0.5$, but not for $\delta = 0.9999$

In [7]:
results_L_computable_05 = load_problems_with_computable_Lipschitz_constant('SOR_all_instances_0.5')
Lipschitz_possible_only_05 = filter_instances_from_A_not_contained_in_B(results_L_computable_05,results_L_computable)
results_L_computable_05.loc[Lipschitz_possible_only_05][['vars','constrs','time L']]

We can compute Lipschitz constants for 197  Problems


,vars,constrs,time L
edgecross14-058,"(183, 182, 182)","(1457, 1, 1, 1)",$520.04
edgecross20-040,"(381, 380, 380)","(4561, 1, 1, 1)",$1.97
edgecross22-048,"(463, 462, 462)","(6161, 1, 1, 1)",$3.33
edgecross24-057,"(553, 552, 552)","(8097, 1, 1, 1)",$14.20
faclay20h,"(191, 190, 190)","(2281, 1, 1, 1)",$147.93
sporttournament30,"(436, 435, 435)","(1, 1, 1, 1)",$10.39
sporttournament32,"(497, 496, 496)","(1, 1, 1, 1)",$14.07
sporttournament34,"(562, 561, 561)","(1, 1, 1, 1)",$87.50
sporttournament36,"(631, 630, 630)","(1, 1, 1, 1)",$154.60
sporttournament38,"(704, 703, 703)","(1, 1, 1, 1)",$283.53


### Feasible points
Let us now investigate the problems for which we get a feasible point. We found that several problems which are originally unconstrained problems, but have been remodelled using the epigraph reformulation. We leave out such problems in our analysis, as the point obtained by FRA-SOR then corresponds to solving the unconstrained problem and rounding the optimal point. 

In [9]:
results_pseudo_granular

,vars,constrs,time L,time SOR,obj,constr_value
autocorr_bern20_03,"(21, 20, 20)","(1, 1, 1, 1)",$0.11,$0.05,$68.01,$-132.01
autocorr_bern25_03,"(26, 25, 25)","(1, 1, 1, 1)",$0.12,$0.04,$62.02,$-150.02
cvxnonsep_normcon20r,"(40, 10, 0)","(21, 20, 10, 10)",$1.17,$0.05,$-14.65,$-inf
cvxnonsep_normcon30r,"(60, 15, 0)","(31, 30, 15, 15)",$1.77,$0.05,$-14.80,$-inf
ex1223a,"(7, 4, 4)","(9, 4, 4, 0)",$0.00,$0.09,$6.07,$-inf
ex4,"(36, 25, 25)","(30, 25, 25, 0)",$0.00,$0.05,$-6.70,$-inf
genpooling_lee1,"(49, 9, 9)","(82, 20, 0, 0)",$0.00,$0.08,"$-4,309.83",$-inf
genpooling_lee2,"(53, 9, 9)","(92, 30, 0, 0)",$0.00,$0.11,"$-3,759.46",$-inf
genpooling_meyer10,"(394, 187, 187)","(423, 33, 0, 0)",$0.00,$0.17,"$5,504,875.19",$-inf
genpooling_meyer15,"(734, 352, 352)","(768, 48, 0, 0)",$0.00,$0.34,"$6,544,066.67",$-inf


In [10]:
results_nonepi = filter_out_only_epi_problems(results_pseudo_granular)
result_nonepi_post_processed = update_objective_value_epi_problems(results_nonepi)

C:\Users\hg2412\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [269]:
store_FRA_results_for_Bonmin(result_nonepi_post_processed)

### Comparison with bonmin for the above problems

In [12]:
res_bonmin = load_obj('res_bonmin')

In [30]:
overview = pd.concat([result_nonepi_post_processed,res_bonmin['time_bonmin']],axis=1)
order = ['vars','constrs', 'obj','time L', 'time SOR', 'time_bonmin']
modified_overview = overview[order]
modified_overview

,vars,constrs,obj,time L,time SOR,time_bonmin
cvxnonsep_normcon20r,"(40, 10, 0)","(21, 20, 10, 10)",$-14.65,$1.17,$0.05,$0.17
cvxnonsep_normcon30r,"(60, 15, 0)","(31, 30, 15, 15)",$-14.80,$1.77,$0.05,$0.09
ex1223a,"(7, 4, 4)","(9, 4, 4, 0)",$6.07,$0.00,$0.09,$0.09
ex4,"(36, 25, 25)","(30, 25, 25, 0)",$-6.70,$0.00,$0.05,$0.70
genpooling_lee1,"(49, 9, 9)","(82, 20, 0, 0)","$-4,309.83",$0.00,$0.08,$0.62
genpooling_lee2,"(53, 9, 9)","(92, 30, 0, 0)","$-3,759.46",$0.00,$0.11,$1.42
genpooling_meyer10,"(394, 187, 187)","(423, 33, 0, 0)","$5,504,875.19",$0.00,$0.17,$3.33
genpooling_meyer15,"(734, 352, 352)","(768, 48, 0, 0)","$6,544,066.67",$0.00,$0.34,$13.56
ndcc12,"(644, 46, 46)","(237, 46, 0, 0)",$108.11,$0.00,$0.12,$1.81
ndcc13,"(630, 42, 42)","(254, 42, 0, 0)",$107.57,$0.00,$0.25,$6.34


In [44]:
SOR_better= np.where(modified_overview['time L']+ modified_overview['time SOR']< modified_overview['time_bonmin'])[0]
modified_overview.iloc[SOR_better,:]

,vars,constrs,obj,time L,time SOR,time_bonmin
ex4,"(36, 25, 25)","(30, 25, 25, 0)",$-6.70,$0.00,$0.05,$0.70
genpooling_lee1,"(49, 9, 9)","(82, 20, 0, 0)","$-4,309.83",$0.00,$0.08,$0.62
genpooling_lee2,"(53, 9, 9)","(92, 30, 0, 0)","$-3,759.46",$0.00,$0.11,$1.42
genpooling_meyer10,"(394, 187, 187)","(423, 33, 0, 0)","$5,504,875.19",$0.00,$0.17,$3.33
genpooling_meyer15,"(734, 352, 352)","(768, 48, 0, 0)","$6,544,066.67",$0.00,$0.34,$13.56
ndcc12,"(644, 46, 46)","(237, 46, 0, 0)",$108.11,$0.00,$0.12,$1.81
ndcc13,"(630, 42, 42)","(254, 42, 0, 0)",$107.57,$0.00,$0.25,$6.34
ndcc14,"(864, 54, 54)","(305, 54, 0, 0)",$143.31,$0.00,$0.20,$41.07
ndcc15,"(680, 40, 40)","(306, 40, 0, 0)",$102.42,$0.00,$0.14,$4.51
ndcc16,"(1080, 60, 60)","(377, 60, 0, 0)",$145.89,$0.00,$0.23,$42.33


In [48]:
print("Sor valuable in", len(modified_overview.iloc[SOR_better,:]), 
      "cases, Bonmin better in", len(modified_overview) - len(modified_overview.iloc[SOR_better,:]), "cases")

Sor valuable in 28 cases, Bonmin better in 11 cases
